In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

import numpy as np

from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from scipy.sparse import hstack

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

lengthTestData = len(test_data)

In [3]:
# Combine text data for preprocessing
text = pd.concat([train_data['text'], val_data['text']], ignore_index=True)
text_test = test_data['text']

In [4]:
linguistic_features = pd.concat([train_data.iloc[:,-31:], val_data.iloc[:,-31:]], ignore_index=True)
test_linguistic_features = test_data.iloc[:,-31:]

In [5]:
linguistic_features

,Words_per_Sentence,Percentage_First_Person_Singular,Percentage_Third_Person,Percentage_Exclusive,Percentage_Negation,Percentage_Causation,Percentage_Sense,Percentage_PositiveEmo,Percentage_NegativeEmo,Percentage_AffectiveTerms,...,Percentage_PastVerb,Percentage_PresentVerb,Percentage_FutureVerb,Percentage_Article,Percentage_Pronoun,Percentage_Conjunction,Lexical_Diversity,Emotiveness,Rate_of_Adjectives_Adverbs,Flesch_Kincaid
0,28.777778,0.000000,1.930502,0.000000,0.000000,2.702703,5.405405,3.474903,3.474903,6.949807,...,3.861004,4.633205,0.772201,5.019305,4.247104,5.405405,0.548263,0.247059,0.081081,14.1
1,32.900000,0.000000,2.735562,1.519757,0.607903,1.519757,4.863222,2.735562,0.607903,3.343465,...,2.127660,2.127660,0.911854,7.294833,5.471125,5.167173,0.595745,0.296610,0.106383,13.5
2,30.416667,0.000000,2.191781,0.821918,0.547945,1.095890,6.027397,3.013699,0.000000,3.013699,...,3.835616,3.013699,1.095890,8.219178,4.931507,3.561644,0.495890,0.246479,0.095890,11.8
3,21.254545,0.085543,4.790419,1.710864,1.197605,2.138580,3.079555,2.053037,1.796407,3.849444,...,2.138580,4.790419,0.427716,6.843456,8.297690,5.560308,0.437981,0.415254,0.125749,9.9
4,24.444444,0.000000,3.181818,1.636364,0.545455,2.181818,6.181818,3.545455,1.727273,5.272727,...,2.818182,5.818182,0.818182,9.000000,5.909091,4.727273,0.440000,0.273171,0.101818,10.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54714,56.166667,0.296736,4.154303,1.186944,1.186944,0.593472,5.341246,2.670623,2.077151,4.747774,...,2.373887,5.044510,0.296736,8.011869,7.715134,4.154303,0.554896,0.274336,0.091988,12.0
54715,22.900000,0.000000,2.183406,0.873362,0.436681,0.000000,5.240175,1.310044,3.493450,4.803493,...,3.056769,3.493450,0.000000,6.113537,4.366812,2.620087,0.620087,0.271605,0.096070,10.7
54716,18.095238,1.842105,5.263158,2.105263,1.052632,1.315789,5.789474,3.684211,0.789474,4.736842,...,5.789474,4.736842,0.263158,4.736842,18.947368,5.789474,0.484211,0.362832,0.107895,6.8
54717,16.611354,1.156677,2.970557,2.602524,1.445846,1.393270,2.628812,2.523659,1.919033,4.468980,...,3.548896,5.783386,0.578339,6.887487,10.778128,6.834911,0.289432,0.457865,0.128549,8.2


In [6]:
test_linguistic_features

,Words_per_Sentence,Percentage_First_Person_Singular,Percentage_Third_Person,Percentage_Exclusive,Percentage_Negation,Percentage_Causation,Percentage_Sense,Percentage_PositiveEmo,Percentage_NegativeEmo,Percentage_AffectiveTerms,...,Percentage_PastVerb,Percentage_PresentVerb,Percentage_FutureVerb,Percentage_Article,Percentage_Pronoun,Percentage_Conjunction,Lexical_Diversity,Emotiveness,Rate_of_Adjectives_Adverbs,Flesch_Kincaid
0,25.066667,0.000000,2.393617,1.595745,0.000000,2.393617,7.446809,2.393617,1.861702,4.255319,...,6.648936,3.191489,0.797872,11.170213,5.585106,5.585106,0.486702,0.345133,0.103723,9.9
1,23.333333,0.000000,2.857143,2.857143,2.857143,2.142857,2.857143,2.142857,0.000000,2.142857,...,7.857143,1.428571,2.142857,5.000000,9.285714,1.428571,0.742857,0.137255,0.050000,12.0
2,24.333333,0.000000,1.369863,1.369863,1.369863,0.000000,8.219178,0.000000,0.000000,0.000000,...,4.109589,0.000000,1.369863,6.849315,1.369863,1.369863,0.698630,0.181818,0.054795,7.7
3,27.466667,0.404531,3.559871,0.566343,0.485437,1.779935,3.478964,2.831715,1.699029,4.530744,...,4.126214,2.993528,0.485437,8.656958,6.877023,4.773463,0.411812,0.285360,0.093042,11.6
4,40.666667,0.000000,3.278689,0.819672,0.000000,3.278689,6.557377,2.459016,2.459016,4.918033,...,6.557377,3.278689,0.819672,8.196721,4.918033,2.459016,0.672131,0.472222,0.139344,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6076,26.708333,0.468019,4.680187,2.184087,0.936037,1.560062,1.872075,1.560062,2.808112,4.368175,...,3.432137,4.056162,1.092044,4.368175,11.076443,6.240250,0.436817,0.325472,0.107644,10.1
6077,32.700000,0.000000,1.834862,0.305810,0.000000,0.000000,4.587156,0.000000,5.504587,5.504587,...,1.529052,4.892966,0.000000,9.785933,3.975535,4.892966,0.483180,0.405405,0.137615,10.4
6078,25.600000,0.000000,4.427083,0.781250,0.520833,3.125000,2.083333,4.687500,1.562500,6.250000,...,1.822917,5.989583,0.781250,6.510417,5.468750,5.989583,0.476562,0.297101,0.106771,11.2
6079,63.000000,0.000000,5.555556,2.777778,0.000000,0.793651,2.380952,1.190476,1.587302,2.777778,...,2.777778,1.984127,0.793651,8.333333,8.333333,1.984127,0.575397,0.270000,0.107143,26.6


In [7]:
# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(filtered_tokens)

In [9]:
text_preprocessed = text.apply(preprocess_text)

In [10]:
print(text_preprocessed)

0        beirut ( reuters ) - iran military chief met s...
1        hanoi ( reuters ) - top u.s. envoy began two-d...
2        ( reuters ) - four u.s. senator asked senate j...
3        first read morning briefing meet press nbc pol...
4        cairo ( reuters ) - six month egypt election ,...
                               ...                        
54714    lack oversight prof donald trump totally unfit...
54715    tucker carlson responded espn anchor calling p...
54716    getting something nothing rage president profe...
54717    black emanuelle fixed 1976. attila speaking eu...
54718    chaos broke legal american illegal alien clash...
Name: text, Length: 54719, dtype: object


In [11]:
print(text)

0        beirut (reuters) - iran s military chief met w...
1        hanoi (reuters) - a top u.s. envoy began a two...
2        (reuters) - four u.s. senators have asked the ...
3        first read is a morning briefing from meet the...
4        cairo (reuters) - six months before egypt s el...
                               ...                        
54714    this lack of oversight proves that donald trum...
54715    tucker carlson responded to an espn anchor cal...
54716    because getting something for nothing is all t...
54717    black emanuelle fixed all that in 1976. attila...
54718    chaos broke out after legal americans and ille...
Name: text, Length: 54719, dtype: object


In [12]:
# Initialize TF-IDF vectorizer without specifying max_features
count_vectorizer = CountVectorizer()

In [13]:
# Fit and transform the preprocessed text data
count_matrix = count_vectorizer.fit_transform(text_preprocessed)

In [14]:
# Get the number of unique tokens
num_unique_tokens = len(count_vectorizer.get_feature_names_out())
print(num_unique_tokens)

169079


In [15]:
# Re-initialize TF-IDF vectorizer with the determined max_features
count_vectorizer = CountVectorizer(max_features=num_unique_tokens)

In [16]:
#Fit and transform the text data again with the updated max_features
count_matrix = count_vectorizer.fit_transform(text_preprocessed)

# Convert the TF-IDF matrix to a CSR (Compressed Sparse Row) matrix for efficient row-wise operations
csr_count_matrix = csr_matrix(count_matrix)

# Find the row index with the maximum number of filled values
max_features_row_index = csr_count_matrix.getnnz(axis=1).argmax()

# Get the number of features in the document with the most filled values
max_features = csr_count_matrix[max_features_row_index].count_nonzero()

svd = TruncatedSVD(n_components=int(max_features*0.3))
count_matrix = svd.fit_transform(count_matrix)

In [17]:
#dense_tfidf_matrix = tfidf_matrix[:len(train_data)]
#dense_val_tfidf_matrix = tfidf_matrix[len(train_data):len(train_data) + len(val_data)]

# Merging the Validation and Training Data into one for a larger training dataset.
#dense_count_matrix = count_matrix[:len(train_data) + len(val_data)]

In [18]:
# Concatenate linguistic features and TF-IDF matrix horizontally
dense_count_with_linguistic = hstack([count_matrix, csr_matrix(linguistic_features)])

In [19]:
# Convert them into Arrays
train_labels = train_data['label'].values
val_labels = val_data['label'].values

dense_labels = np.concatenate((train_data['label'].values, val_data['label'].values), axis=0)

In [20]:
# Create and train the SVM model
svm_model = SVC(kernel='rbf', C=5.0)
#svm_model = SVC(kernel='poly', C=20.0, degree=2, coef0=0.001)

In [21]:
#CROSS FOLD VALIDATION, DELETE IF NECESSARY

# Initialize KFold with the desired number of folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(svm_model, dense_count_with_linguistic, dense_labels, cv=kf, scoring='accuracy')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Average CV accuracy:", cv_scores.mean())

Cross-validation scores: [0.95458699 0.95769371 0.95860746 0.95422149 0.96097962]
Average CV accuracy: 0.9572178541597529


In [22]:
svm_model.fit(dense_count_with_linguistic, dense_labels)

SVC(C=5.0)

In [23]:
text_test_preprocessed = text_test.apply(preprocess_text)
test_count_matrix = count_vectorizer.transform(text_test_preprocessed)
dense_test_count_matrix = svd.transform(test_count_matrix)
dense_test_count_with_linguistic = hstack([dense_test_count_matrix, csr_matrix(test_linguistic_features)])
test_labels = test_data['label'].values

test_predictions = svm_model.predict(dense_test_count_with_linguistic)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.9602039138299622


In [24]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(test_labels, test_predictions)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      3420
           1       0.95      0.96      0.95      2661

    accuracy                           0.96      6081
   macro avg       0.96      0.96      0.96      6081
weighted avg       0.96      0.96      0.96      6081



In [25]:
from joblib import dump
dump(svm_model, 'svm_model_cv.joblib')

['svm_model_cv.joblib']

In [26]:
test_accuracy*lengthTestData

5839.0